#### **Introduction to Vision Transformer (ViT)**

The [Vision Transformer](https://arxiv.org/abs/2010.11929) (ViT) represents an innovative breakthrough in how machines interpret images. Developed by Google Research, the ViT draws inspiration from the transformative impact of transformer models in Natural Language Processing (NLP). Traditionally, Convolutional Neural Networks (CNNs) dominated image analysis tasks. However, the introduction of ViT marked a paradigm shift, demonstrating that a model primarily designed for text could excel in visual domains as well. The ViT model has not only outperformed existing CNN-based methods across various image classification benchmarks but also shown remarkable scalability and generalization capabilities. Its success has led to its adoption in several leading-edge applications, including image segmentation in [Segment Anything](https://arxiv.org/abs/2304.02643) (SAM) and multimodal learning in [Contrastive Language-Image Pre-training](https://arxiv.org/pdf/2103.00020.pdf) (CLIP).

This notebook provides an introduction to the Vision Transformer (ViT) model, explaining its architecture and how it can be implemented using PyTorch for image classification tasks.

#### **CIFAR-10 Dataset**

The CIFAR-10 dataset will serve as our primary playground for experimenting with ViT.

Comprising 60,000 images, each 32x32 pixels and in full color, CIFAR-10 is categorized into 10 distinct classes, with each class containing 6,000 images. The dataset is divided into a training set of 50,000 images and a testing set of 10,000 images.

Use the following code snippet to download and prepare the data:

In [ ]:
from torchvision import transforms, datasets

train_val_dataset = datasets.CIFAR10(root="./data", train=True, download=True, 
                                     transform=transforms.ToTensor())
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, 
                                transform=transforms.ToTensor())

This script initializes the CIFAR-10 dataset for training and testing purposes using PyTorch's `datasets.CIFAR10` class for seamless data handling. 

**(1)** To initiate the training dataset, the script specifies the root directory for storing the data, sets the `train` parameter to `True`, and enables the download option. Additionally, it uses the `transforms.ToTensor()` method to convert the images into PyTorch tensors.

**(2)** To initialize the testing dataset, similar parameters are used except for the `train` parameter, which is set to `False`.

Let's visualize images from the CIFAR-10 dataset by defining the function `plot_class_examples`.

In [2]:
import matplotlib.pyplot as plt

def plot_class_examples(dataset, n_images):
    """Plot the first images for each class in the dataset."""
    classes = dataset.classes
    class_img_dict = {c: [] for c in range(len(classes))}
    for i, (_, label) in enumerate(dataset):
        if all(len(v) == n_images for v in class_img_dict.values()): 
            break
        if len(class_img_dict[label]) < n_images: 
            class_img_dict[label].append(i)
    
    fig, axs = plt.subplots(n_images, len(classes), figsize=(len(classes), 3))
    for class_idx, img_indices in class_img_dict.items():
        for j, img_index in enumerate(img_indices):
            img = dataset[img_index][0].permute(1, 2, 0)
            axs[j, class_idx].imshow(img)
            axs[j, class_idx].set(xticks=[], yticks=[])
            if j == 0: 
                axs[j, class_idx].set_title(classes[class_idx], size="medium")
    plt.show()

This function displays the first  `n_images` examples for each class from the dataset.

**(1)** The process starts by extracting the classes from the dataset and **(2)** initializing a dictionary to hold indices of images for each class. **(3-4)** Then, as it iterates over the dataset, the function checks whether enough images have been collected for each class. **(5-6)** If a class still needs more images, the current index is added to its list of indices. **(7)** Once the desired number of images for each class is reached, the function prepares a matplotlib figure with a grid of subplots. The subplots are arranged to have one row per class and one column per image. This ensures that the layout supports the visualization of all selected images. 

**(8-10)** For each class, the function iterates over its collected image indices, retrieves each image, permutes it for correct display formatting, and plots them in the corresponding subplot cell.

In [ ]:
plot_class_examples(train_val_dataset, n_images=3)

#### **Preparing the data**

Preparing the data involves more than just loading it; we also need to ensure that our model trains effectively and generalizes well to unseen data. To this end, the CIFAR-10 dataset, initially comprising only training and testing subsets, requires further partitioning to create a separate validation set.

For this purpose, we implement the `split_train_val` function, which splits the training set into training and validation subsets given a specified validation split ratio. This split allows us to independently evaluate the model's performance and make adjustments as needed without compromising the integrity of the test dataset.


In [4]:
import copy, torch
from torch.utils.data import Subset

indices = torch.randperm(len(train_val_dataset)).tolist()
split = int(len(train_val_dataset) * 0.20)
train_indices, val_indices = indices[split:], indices[:split]
train_dataset = Subset(copy.deepcopy(train_val_dataset), train_indices)
val_dataset = Subset(copy.deepcopy(train_val_dataset), val_indices)

The `split_train_val` function takes a dataset and a validation split ratio as inputs, and returns the training and validation subsets. The function performs the following steps:

**(1)** It generates random indices for the dataset.

**(2)** It calculates the split index based on the validation split ratio.

**(3)** It splits the indices into training and validation indices.

**(4)** It creates the training subset using the `Subset` class, which allows us to create a subset of the dataset using the specified indices.

**(5)** It creates the validation subset using the same method as step 4. To ensure that the training and validation subsets are independent of each other, the function uses the `copy.deepcopy` function.

For this example, we will use a validation split ratio of 0.2, which means that 20% of the training data will be reserved for validation.

To enhance the model's ability to generalize and prevent overfitting, we will apply various data augmentation techniques to the training dataset. 

In [5]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(), transforms.ToTensor(),
    transforms.Normalize(mean=[0.49139968, 0.48215841, 0.44653091], 
                         std=[0.24703223, 0.24348513, 0.26158784]),
])
train_dataset.dataset.transform = train_transform

These include **(1)** random cropping and padding, **(2)** horizontal flipping, and **(3)** normalization of the image tensors using the CIFAR-10 training set mean and standard deviation values for each channel.

For the validation dataset, however, we'll restrict the preprocessing to just conversion to tensors and normalization, avoiding augmentation to maintain the validity of the evaluation process.

In [6]:
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.49139968, 0.48215841, 0.44653091], 
                         std=[0.24703223, 0.24348513, 0.26158784]),
])
val_dataset.dataset.transform = val_transform

#### **Defining data loaders**

Next, we will define data loaders that will feed the training, validation, and test data to the model during the training and evaluation processes.

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False)

#### **Building a Vision Transformer model**

Having prepared our dataset, we define the Vision Transformer (ViT) model. 

**Figure 1** illustrates the ViT architecture. It is built around four pivotal components that work together to process and classify images.

Firstly, **Image Patch Embeddings** play an essential role by slicing the input image into fixed-size, non-overlapping patches. These patches are linearly embedded, creating a series of flattened embeddings that serve as the input sequence for the transformer encoder (**Figure 1a**). To capture global information about the image, an additional learnable class token is integrated with these patch embeddings (**Figure 1b**). This class token, while not directly corresponding to any individual patch, symbolizes the collective image and its interactions with the embeddings, providing a holistic view for classification.

Secondly, the model incorporates **Positional Encodings** to permeate the patch embeddings with spatial context (**Figure 1b**). Unlike sin/cos encoding, these positional encodings are initially randomized and subsequently refined through the learning process, ensuring the model comprehends the layout and position of each patch within the image.

The **Transformer Encoder**, the third component, is the heart of the model (**Figure 1c**). It comprises several layers of transformer blocks that each feature a multi-head self-attention mechanism and a feedforward neural network. This structure allows the model to focus on different parts of the image simultaneously, facilitating a deeper understanding of the visual content.

The **Classification Head** or **Dense Top** is the last component of ViT (**Figure 1d**). It takes the class token output from the transformer encoder and feeds it through a feedforward neural network to predict the image's class label.


<p align="center">
  <img src="images/vit-image.png" width=600 height=auto>
  <br>
  <b>Figure 1. Vision Transformer (ViT) Architecture</b>
</p>

Let us now instantiate the ViT model

In [8]:
import deeplay as dl

vit = dl.ViT(in_channels=3, image_size=32, patch_size=4, 
             hidden_features=[384,] * 7, out_features=10, num_heads=12)

The Deeplay `ViT` class is responsible for managing the ViT (Vision Transformer) model architecture. The class constructor initializes the model components, including the patch embeddings, positional encodings, transformer encoder, and classification head. `ViT` accepts the following parameters:

**(1)** `in_channels`: This parameter defines the number of input channels in the image. For the CIFAR-10 dataset, the number of channels is 3 (RGB).

**(2)** `image_size`: This parameter defines the size of the input image, which is 32x32 for the CIFAR-10 dataset.

**(3)** `patch_size`: This parameter defines the size of the image patches. By dividing the image into smaller patches, the model can analyze the image in segments and learn local features before integrating them for global understanding. This parameter is set to 4, resulting in a total of 64 patches of size 4x4.

**(4)** `hidden_dim`: This parameter defines the dimension of the hidden layers in the transformer encoder. Seven transformer blocks (or layers) are defined in the model, each with a hidden size of 384 channels.

**(5)** `out_channels`: This parameter defines the number of classes in the dataset, which is 10 for CIFAR-10.

**(6)** `num_heads`: This parameter defines the number of attention heads in the multi-head self-attention mechanism. In this case, it is set to 12.


In [ ]:
print(vit)

#### **Training the model**

The following create a `Trainer` object to train the model.

We will use the Adam optimizer with a learning rate of `1e-3` and weight decay of `1e-5`, and `CrossEntropyLoss` as the loss function.

In [ ]:
classifier = dl.Classifier(
    model=vit, num_classes=10,
    optimizer=dl.Adam(lr=1e-3, weight_decay=5e-5, betas=(0.9, 0.999)),
).build()
trainer = dl.Trainer(max_epochs=100)

In [ ]:
trainer.fit(classifier, train_dataloader, val_dataloader)
trainer.history.plot()

Use the following code snippet to visualize the training and validation loss and accuracy.

The ViT model achieves a validation accuracy of approximately 76% and reaches saturation at that point. While this is not a bad performance, it is still far from being comparable to the classification ability of Convolutional Neural Networks.

On the test set, the model achieves a similar accuracy:

In [12]:
test_dataset.transform = val_transform
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
trainer.test(classifier, test_loader)

However, the ViT model can be further fine-tuned to improve its performance.

#### **Improving the model**

Compared to CNNs, ViT models lack strong inductive biases. Inductive bias refers to the set of assumptions a model makes about the data it trains on to generalize from the training data to unseen data effectively. For instance, CNNs inherently assume spatial hierarchies in images, leveraging local patterns (e.g., edges and textures in early layers) and composing them into higher-order features (like objects) in deeper layers. This architectural  innatency towards capturing spatial locality and translation invariance directly supports learning from image data.

Due to the absence of these biases, ViT requires a substantial amount of data to learn effectively. It treats the image as a sequence of patches and learns to interpret these without the innate advantage of assuming spatial correlations, relying instead on the data to dictate these relationships. Consequently, ViT models need larger datasets to achieve their impressive performance, as they must learn from scratch what CNNs get for free through their architecture.

The CIFAR-10 dataset, containing only 60,000 images, is relatively small compared to the massive datasets where ViT models excel. Although 60,000 might seem substantial, it pales in comparison to the vast datasets like ImageNet with over 14 million images, which have been instrumental in training ViTs to their state-of-the-art performance.

[CutMix](https://arxiv.org/abs/1905.04899) is a technique designed to address this challenge, among others. It works by forcing the network to learn local relationships more efficiently, effectively injecting the network with sufficient pathways to learn spatial insights without relying on vast amounts of data.

#### **Implementing CutMix**

CutMix is a data augmentation strategy that seeks to enhance the model's ability to understand and integrate local features within images, thereby improving its generalization capabilities. Originating from the idea of [Mixup](https://arxiv.org/abs/1710.09412), where two images are blended together by averaging their pixels, CutMix takes this concept further by combining patches from two different images rather than mixing their entire content. This technique not only enriches the dataset but also introduces a more challenging and nuanced learning task for the model.

In a typical CutMix  augmentation, a patch from one image is cut and pasted onto another image, and the labels are mixed proportionally to the area of the patches. This process creates a new training example that is a composite of features and labels from two distinct images. For the model, the task then becomes to predict a mixed label, reflecting the composition of the input image. The primary advantage of this method is its ability to force the model to focus on less dominant features of the image, promoting a deeper and more robust learning of spatial relationships and feature representations.

The following class `CutMix` implements the CutMix augmentation.

In [14]:
import numpy as np

class CutMix(object):
    """CutMix."""

    def __init__(self, size, beta):
        """Initialize CutMix."""
        self.size, self.beta = size, beta

    def __call__(self, batch):
        """Execute CutMix."""
        imgs, labels = batch
        rand_idx = torch.randperm(imgs.size(0))
        rand_imgs, rand_labels = imgs[rand_idx], labels[rand_idx]
        initial_mix_ratio = np.random.beta(self.beta, self.beta)
        augmented_imgs = imgs.clone()
        
        r_x = np.random.uniform(0, self.size)
        r_y = np.random.uniform(0, self.size)
        r_w = self.size * np.sqrt(1 - initial_mix_ratio)
        r_h = self.size * np.sqrt(1 - initial_mix_ratio)

        x1 = np.clip(int(r_x - r_w // 2), a_min=0, a_max=self.size)
        x2 = np.clip(int(r_x + r_w // 2), a_min=0, a_max=self.size)
        y1 = np.clip(int(r_y - r_h // 2), a_min=0, a_max=self.size)
        y2 = np.clip(int(r_y + r_h // 2), a_min=0, a_max=self.size)

        augmented_imgs[:, :, y1:y2, x1:x2] = rand_imgs[:, :, y1:y2, x1:x2]
        final_mix_ratio = 1 - ((x2 - x1) * (y2 - y1) / (self.size ** 2))
        return augmented_imgs, labels, rand_labels, final_mix_ratio

Let's visualize the CutMix augmentation by applying it to a random pair of images from the training dataset.

In [ ]:
import random
from itertools import chain

def create_batch(samples):
    """Create batch."""
    images, labels = zip(*samples)
    return torch.stack(images), torch.tensor(labels)

def normalize_image(image):
    """Normalize image."""
    mean = torch.tensor([0.49139968, 0.48215841, 0.44653091]).view(3, 1, 1)
    std = torch.tensor([0.24703223, 0.24348513, 0.26158784]).view(3, 1, 1)
    return image * std + mean

samples = random.choices(train_dataset, k=2)
batch = create_batch(samples)

augmented_imgs, *_, mix_ratio = CutMix(size=32, beta=1.0)(batch)

fig, axs = plt.subplots(1, 4, figsize=(12, 3))
for i, img in enumerate(chain(batch[0], augmented_imgs)):
    axs[i].imshow(normalize_image(img).permute(1, 2, 0))
    axs[i].set_title("Original" if i < 2 else f"CutMix {mix_ratio:.2f}")
    axs[i].axis("off")    
plt.savefig("fig_08_C4.pdf", bbox_inches="tight")  ### plt.show()

This code snippet **(1)** selects two random images from the dataset `train_dataset`. Next, **(2)** the `create_batch` function is used to create a batch of images and labels from the selected images. **(3)** The `CutMix` class is then applied to the batch to generate the augmented images and labels. Finally, **(4)** the original and augmented images are displayed side by side for comparison.

*Please note that due to possibility of the shuffle operation resulting in the original order, there's a chance that the images might appear identical in certain instances. If this occurs, consider re-running the operation.*

#### **Training with CutMix**

To train the model with CutMix, we define the `CutMixClassifier` class, which extends the `Classifier` to incorporate the CutMix augmentation.

In [16]:
from torch.optim.lr_scheduler import CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler as GradualWarmup

class CutMixClassifier(dl.Classifier):
    """Classifier with CutMix."""

    def __init__(self, model, size=32, beta=1.0, **kwargs):
        """Initialize classifier with CutMix."""
        super().__init__(model, **kwargs)
        self.cutmix = CutMix(size=size, beta=beta)

    def training_step(self, batch, batch_idx):
        """Perform one training step."""
        augmented_imgs, labels, rand_labels, mix_ratio = self.cutmix(batch)
        pred_labels = self.model(augmented_imgs)
        loss = (mix_ratio * self.loss(pred_labels, labels) 
                + (1 - mix_ratio) * self.loss(pred_labels, rand_labels))

        self.log("train_loss", loss, on_step=True, on_epoch=True,
                 prog_bar=True, logger=True)
        self.log_metrics("train", pred_labels, labels, on_step=True, 
                         on_epoch=True, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        """Configure optimizers."""
        optimizer = super().configure_optimizers()
        base_scheduler = CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-5)
        scheduler = GradualWarmup(optimizer, multiplier=1.0, total_epoch=5, 
                                  after_scheduler=base_scheduler)
        return {"optimizer": optimizer,
                "lr_scheduler": {"scheduler": scheduler, "interval": "epoch"}}

During training,  **(1)** `CutMixClassifier` first applies CutMix augmentation to the input batch, creating augmented images with their corresponding labels and random labels alongside the mixing coefficient `lam`. **(2)** The model processes the augmented images, and **(3)** the loss is calculated using a combination of the original and random labels weighted by `lam` and `1-lam`, respectively, to reflect the proportion of each image in the augmented mix. 

To further enhance the performance of the model,  `CutMixClassifier` **(4)** sets up a learning rate schedule starting with a `CosineAnnealingLR` scheduler to adjust the learning rate following a cosine curve, and then **(5)** employs a warm-up phase using `GradualWarmupScheduler`,  which gradually increases the learning rate over the first five epochs to the initial learning rate before transitioning to the cosine annealing schedule. This approach optimizes the learning process, improving training stability and model performance.

Now we proceed to train the model using the `CutMixClassifier` class. 

The model is trained for 700 epochs to allow sufficient time for the model to learn from the augmented data and improve its performance.

In [ ]:
vit_cutmix = dl.ViT(in_channels=3, image_size=32, patch_size=4, 
                    hidden_features=[384,] * 7, out_features=10, num_heads=12)
vit_cutmix[..., "attention#-1"].log_output("attention_output")

classifier_cutmix = CutMixClassifier(
    model=vit_cutmix, num_classes=10,
    optimizer=dl.Adam(lr=1e-3, weight_decay=5e-5, betas=(0.9, 0.999)),
).build()
trainer_cutmix = dl.Trainer(max_epochs=1)  ### trainer_cutmix = dl.Trainer(max_epochs=700)
trainer_cutmix.fit(classifier_cutmix, train_dataloader, val_dataloader)

**(1)** the `log_output` method is used to log the output of the attention layer in the last transformer block. This will allow us to visualize the attention maps generated by the model during inference for interpretability purposes.

Note that the model has achieved a validation accuracy of approximately 90%, which is a significant improvement over the previous accuracy of 76%.

In [ ]:
trainer_cutmix.test(classifier_cutmix, test_loader)

In [ ]:
classifier_cutmix.logs

#### **Visualizing attention maps**

Let's visualize the model's performance by plotting test images, attention maps and predicted labels.

In [ ]:
from skimage.transform import resize

samples = random.choices(test_dataset, k=4)
imgs, labels = create_batch(samples)

classifier_cutmix.model.eval()
output = classifier_cutmix(imgs)
preds = torch.argmax(output, dim=1)

attn_maps = \
    classifier_cutmix.logs["attention_output"][1][:, 0, 1:].reshape(4, 8, 8)

fig, axs = plt.subplots(2, 4, figsize=(12, 6))
for i, (img, label, pred, attn_map) in enumerate(
    zip(imgs, labels, preds, attn_maps)
):
    img = normalize_image(img).detach().permute(1, 2, 0)

    axs[0, i].imshow(img)
    axs[0, i].set_title(f"Label: {test_dataset.classes[label]}\n"
                        + f"Pred: {test_dataset.classes[pred]}")
    axs[0, i].axis("off")

    resized_attn_map = resize(attn_map.detach().numpy(), (32, 32), 
                              anti_aliasing=True)
    axs[1, i].imshow(img)
    axs[1, i].imshow(resized_attn_map, cmap="hot", alpha=0.5)
    axs[1, i].set_title("Attention Map")
    axs[1, i].axis("off")
plt.show()

#### **Using a pre-trained ViT model**

The model achieved a 90% accuracy, which is comparable to the performance levels of CNNs on the CIFAR-10 dataset. However, as the original ViT paper states, this model achieves its best performance when trained on large-scale datasets like ImageNet, which contains millions of images. In such cases, the model can leverage its capacity to learn from vast amounts of data and generalize effectively, outperforming CNNs significantly.

To demonstrate this, we will use a pre-trained ViT model that has already learned rich representations from a diverse range of images in the ImageNet dataset. This model can perform well on a variety of tasks. 

The following loads the pre-trained ViT model and wrap it in a Torch Module that appends a classification head.

In [31]:
from transformers import ViTModel

class PretrainedViTModel(torch.nn.Module):
    """Pretrained ViT model."""

    def __init__(self, pretrained_model, output_channels):
        """Initialize pretrained ViT model."""
        super().__init__()
        self.backbone = ViTModel.from_pretrained(pretrained_model)
        hidden_features = self.backbone.config.hidden_size
        self.classifier = torch.nn.Linear(hidden_features, output_channels)

    def forward(self, imgs):
        """Perform forward step."""
        features = self.backbone(imgs).last_hidden_state[:, 0]
        return self.classifier(features)

Images from ImageNet are typically larger than those in CIFAR-10, so we need to resize the images to 224x224 before feeding them to the model. We also normalize the images using the mean and standard deviation values of the ImageNet dataset.

In [36]:
import torchvision.transforms as trans_pt
from transformers import ViTImageProcessor

pretrained_model = "google/vit-base-patch16-224-in21k"
processor = ViTImageProcessor.from_pretrained(pretrained_model)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

train_trans_pt = [
    trans_pt.Resize((size, size)), trans_pt.RandomCrop(size, padding=4),
    trans_pt.RandomHorizontalFlip(), trans_pt.ToTensor(),
    trans_pt.Normalize(image_mean, image_std),
]

val_trans_pt = [
    trans_pt.Resize((size, size)), trans_pt.ToTensor(),
    trans_pt.Normalize(image_mean, image_std),
]

train_dataset_pt = copy.deepcopy(train_dataset)
train_dataset_pt.dataset.transform = transforms.Compose(train_trans_pt)
val_dataset_pt = copy.deepcopy(val_dataset)
val_dataset_pt.dataset.transform = transforms.Compose(val_trans_pt)
test_dataset_pt = copy.deepcopy(test_dataset)
test_dataset_pt.transform = transforms.Compose(val_trans_pt)

We will redefine the data loaders to accommodate the resizing and normalization of the images.

In [37]:
train_dataloader_pt = DataLoader(train_dataset_pt, batch_size=32, shuffle=True)
val_dataloader_pt = DataLoader(val_dataset_pt, batch_size=32, shuffle=False)
test_dataloader_pt = DataLoader(test_dataset_pt, batch_size=32, shuffle=False)

and train the model using the `Classifier` class.

In [ ]:
classifier_pt = dl.Classifier(
    model=PretrainedViTModel(pretrained_model, output_channels=10), 
    num_classes=10, optimizer=dl.Adam(lr=2e-5), 
).create()
trainer_pt = dl.Trainer(max_epochs=2)
trainer_pt.fit(classifier_pt, train_dataloader_pt, val_dataloader_pt)

After only two epochs, the pre-trained ViT model achieves an accuracy of approximately 98% on the CIFAR-10 dataset, outperforming the performance of the model trained from scratch.

In [ ]:
trainer_pt.test(classifier_pt, test_dataloader_pt)